In [1]:
import requests, json, logging, time, urllib
import pandas as pd
from io import StringIO
import io as stringIOModule
from datetime import datetime
from optparse import OptionParser
import configparser



In [3]:
# Set up logging to catch errors, etc...

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler(str(time.strftime("%d_%m_%Y")) +"_looker_API_Calls" + ".log")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info('Testing Logs')

# Connect to Looker
LookerAPI class contains information on connecting to Looker via the API. 

The calls available via this api class allow you to 
1. Establish a connection to the API 
2. Get Look Results (run_look)
3. Get Query Results based on Query Slug (run_query_slug)

In [3]:
class LookerAPI(object):
    """Class to contain methods and variables related to looker API authentication and Requests
    """
    def __init__(self, api_endpoint, client_id, client_secret, login_url):
        self.api_endpoint = api_endpoint
        self.client_secret = client_secret
        self.client_id = client_id
        self.login_endpoint = login_url
#         print(self.login_endpoint)
        
    def login(self):
        """login to looker API"""
        try:
            auth_data = {'client_id':self.client_id, 'client_secret':self.client_secret}
            r = requests.post( self.login_endpoint,data=auth_data) # error handle here
            json_auth = json.loads(r.text)['access_token']
            return json_auth
        except requests.exceptions.RequestException as e:
            logger.error(e)

    def run_look(self, look_id, json_auth,return_format='csv'):
        """run look and return as csv, need to add more formats here"""
        try:
            look_run_url = self.api_endpoint + '/looks/{0}/run/{1}'.format(look_id,return_format)
            #r = requests.get(look_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(look_run_url + '?' + 'access_token=' + json_auth)
            return r.text
        except requests.exceptions.RequestException as e:
            logger.error(e)
            
    def run_query(self, query_id, json_auth, return_format='csv'):
        """run query and return as csv, need to add more formats here"""
        try:
            query_run_url = self.api_endpoint + '/queries/{0}/run/{1}'.format(query_id,return_format)
            #r = requests.get(query_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(query_run_url + '?' + 'access_token=' + json_auth)
            return r.text
        except requests.exceptions.RequestException as e:
            logger.error(e)
            
    def run_query_slug(self, query_slug, json_auth):
        """run query and return as csv, need to add more formats here"""
        try:
            query_slug_run_url = self.api_endpoint + '/queries/slug/{0}'.format(query_slug)
            #r = requests.get(query_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(query_slug_run_url + '?' + 'access_token=' + json_auth)
            qid=json.loads(r.text)["id"]
            print("Query_id: " + str(qid))
            return LookerAPI.run_query(self, qid, json_auth)
        except requests.exceptions.RequestException as e:
            logger.error(e)

## Test  LookerAPI calls 

Test out API calls on the demonew looker instance! Modify the look ID, Query ID,  or the Query Slug you'd like results from. 

In order to make an API Call, first establish a connection with the Looker Application. Establishing a connection returns a token, that will be used in any subsequent API calls. 

In [4]:
# Set API credentials for IBM Looker Instance
api_endpoint = 'https://ibm-wdai.looker.com:19999/api/3.0/'
login_url = 'https://ibm-wdai.looker.com:19999/login'
client_id = 'vdRvj8tp8qSC4Yyt3ttj'
client_secret = 'gVYw9wQXY6n85fJCqyf5ZBqc'
demo = LookerAPI(api_endpoint, client_id, client_secret, login_url)

# Login to Instance - Return token to be used in subsequent calls.
json_auth = demo.login()

print('Token:' + json_auth)

# Get Results of Look and Query ID (QID)
look_data = demo.run_look(4,json_auth)
print(look_data)

# # Get Results of Look by passing in the Query Slug. The slug can be found in the URL. 
query_response = demo.run_query_slug("BeQfwKO07hwbMBqtwoyXtU", json_auth)
print(query_response)

# Convert Query Response from CSV to a dateframe
records = pd.read_csv(StringIO(query_response))
# records.head(5)


Token:Ns4xcndk42nR4XZcSr3wKPNdPVRbfRT3jNy5XyRq
Trip Trip Count
286858

Query_id: 175
Trip Trip Count
286858




So far we've sucessfully established a connection to Looker that allows us to pull in modeled data from the Looker explore page or a saved look. 

Lets take that data and use it to build a predictive Model. 

# Example 1

Let's see how different Weather factors for a particular day affect the number of trips taken. 
How does the temperature or the humidity on any given day affect the average trip length?

### Step 1:
Pass in Data from Looker via API calls
An Example Query might look like this -- 
https://ibm-wdai.looker.com/looks/5

Temperature: cJhH3ne0pJVL0yJ4mfi27v or https://ibm-wdai.looker.com/looks/5 <br/>
Humidity: ICgSezXao5VNYvJJioWWA2 or https://ibm-wdai.looker.com/looks/6 <br/>
Temperature + Humidity: 1dcKd5LCfeSo72onWGlCJv or https://ibm-wdai.looker.com/looks/7 <br/>


### Step 2: 
Build a Predictive Model (In our example we use a Linear Regression model).
Our regression model uses the Stats Library and prints out the coefficients, Rsquare, P-value, standard deviation, etc... <br/>
```
smf.ols(formula='Y ~ X', data=data).fit()
```

### Step 3:
Iterate through the Model till you are satisfied with the R2 score. Pass in different fields from Looker via the query slug to predict trip time based on different factors. 

In [5]:
# Import Stats packages 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

# Step 1
# Set API Credentials for Demo Looker Instance and make an API call to get query results. 
api_endpoint = 'https://ibm-wdai.looker.com:19999/api/3.0/'
login_url = 'https://ibm-wdai.looker.com:19999/login'
client_id = 'vdRvj8tp8qSC4Yyt3ttj'
client_secret = 'gVYw9wQXY6n85fJCqyf5ZBqc'

demo = LookerAPI(api_endpoint, client_id, client_secret, login_url)

# Login to Instance - Return token to be used in subsequent calls.
# Run a query with date, temp, humidity, and trip count to use to build regression
# https://ibm-wdai.looker.com/explore/bike_trips/trip?qid=1dcKd5LCfeSo72onWGlCJv
try:
    json_auth = demo.login()
    query_response = demo.run_query_slug("1dcKd5LCfeSo72onWGlCJv", json_auth)
except:
    logger.warn("exception logging in and/or running query!")
    
# Convert Query Response from CSV to a dateframe
records = pd.read_csv(StringIO(query_response))
data = pd.DataFrame(records)

print(data.head())

# Step 2
# Set Predictors and Responses
try:
    column_list = list(data)
    if(len(column_list) > 2 or len(column_list) < 6):
        print("Columns: " + str(column_list))
except:
    logger.error("Your Query doesn't seem right. Recreate Query in Looker.")
        

# Last Item on the List = Dependent variable, Remaining items = part of Predictor
Y = data[column_list.pop()] # Response ---> Average Trip Duration in Minutes
column_list.pop(0)
X = data[column_list] # Predictor ---> All remaining columns are used to predict the model. 
print("X Values: " + str(column_list))

# Apply a Fitted Model to the dateframe
model = smf.ols(formula='Y ~ X', data=data).fit()
print(model.summary())

# Regression Coefficients of the Model are stored in the Params field
trip_count_df = pd.DataFrame(model.params)
trip_count_df = trip_count_df.T
trip_count_df["Predictor"] = str(column_list)

print(list(trip_count_df))
columns_list = [re.sub('[^A-Za-z0-9]+', '', col) for col in list(trip_count_df)]
# colnames(trip_time_df) <- cols
trip_count_df.columns = columns_list

trip_count_df.head()




# Step 3
# Rerun Steps 1 and 2 with different parameters if necessary


Query_id: 188
  Trip Start Date  Weather Temperature (F)  Weather Humidity  Trip Trip Count
0      2016-08-31                       65                77              319
1      2016-08-30                       64                69              375
2      2016-08-29                       68                65              369
3      2016-08-28                       68                65              392
4      2016-08-27                       66                65              333
Columns: ['Trip Start Date', 'Weather Temperature (F)', 'Weather Humidity', 'Trip Trip Count']
X Values: ['Weather Temperature (F)', 'Weather Humidity']
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.686
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     541.7
Date:                Mon, 19 Mar 2018   Prob (F-s

,Intercept,X0,X1,Predictor
0,222.364652,7.74059,-4.628307,"['Weather Temperature (F)', 'Weather Humidity']"


Interpreting the model:
The above graphic describes our model. We can see that: 

1. A 1 degree increase in temperature leads to nearly 8 (7.74059) additional trips
2. Temperature and Humidity has a significant effect on trip count (P > | t| is equal to 0)
3. The R^2 of our basic model is .686 (The model explains 68.6% of the variability in trip count) the AIC is 5914 (We will use this to compare to our models containing other variables)


Step 4: 
Let's persist our predictive model results back to the database. In this example we use the credentials from our Db2 connections in DSX and the ingest.Connectors and pyspark.sql packages to convert the dateframe to a spark dataframe and load to a new table called 'Trip Count Prediction'.

We can leverage the connections feature of Watson Knowledge Studio to automatically include connection credentials for our connected data sources.

In [38]:
# The code was removed by DSX for sharing.

In [39]:
from ingest.Connectors import Connectors
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

dashdbsaveoption = {
Connectors.DASHDB.HOST : credentials["host"],
Connectors.DASHDB.DATABASE : credentials["database"],
Connectors.DASHDB.USERNAME : credentials["username"],
Connectors.DASHDB.PASSWORD : credentials["password"],
Connectors.DASHDB.TARGET_TABLE_NAME : 'BIKE_TRIPS.TRIP_COUNT_PREDICTION',
Connectors.DASHDB.TARGET_TABLE_ACTION : 'replace',
Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' }

trip_count_df2 = sqlContext.createDataFrame(trip_count_df)
NewdashDBDF = trip_count_df2.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

Now that we have our predictive model coefficients, Let's pull it up in Looker and calculate how our actual trip times compared to predictive trip times. 

We do this by joining creating a view called 'trip_count_prediction' that references our new table. We join those coefficients into our existing model and Trip explore.


### Step 5:

Join in predictions table in Looker and create fields to predict the Average trip time based on Weather factors. 

```
explore: trip {
  join: trip_count_prediction {
    type: cross
    relationship: many_to_one
  }
```

**Predictive Fields **
Our predictive measure follows the linear regression model that we used and pulls in the coefficients and intercept that we just persisted back in to the DB.

**Multiple Linear Regression Model:**
$y = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$ <br/>
Each $x$ represents a different feature, and each feature has its own coefficient. <br/>
In our case: <br/>
$y = \beta_0 + \beta_1 \times Temperature + \beta_2 \times Humidity + \beta_3 \times Other Factors$

We calculate the trip count by creating a measure that calculates the regression function and using our coefficients and the temparature and humidity values. 
```
  measure: trip_count_prediction{
    type: average
    sql:  (${trip_count_prediction.x0} * ${weather.temperature}) + 
          (${trip_count_prediction.x1} * ${weather.humidity}) + 
          ${trip_count_prediction.intercept};;
    value_format_name: decimal_1
    view_label: "Trip Count Prediction"
  }
```

Let's compare our actual average trip times vs. those predicted by our model: <br/>
Data Table/Viz: https://ibm-wdai.looker.com/looks/8 <br/>


Lets take this a step further and use our model to forecast average trip times/lengths for future dates based on the same factors we've outlined above (the temperature and the humidity).

We have a seperate notebook that runs daily and pulls the 7 day weather forecast and uploads to a separate table in our Db2 warehouse so we can make future predictions. 

You can access it here: https://dataplatform.ibm.com/analytics/notebooks/v2/3b0de617-210f-4700-9105-9157c2bb8a7a/view?access_token=292635bee35207582e6adceea010e81b1e06386651832e8340e52e299f6920bf





### Step 6: 
Future Predictions: Now that we have the weather forecast for the upcoming week, lets try and predict the trip count for the upcoming week. 

Looker Data/Visualization: https://ibm-wdai.looker.com/looks/14


While this information is useful by itself, we could forecast other factors (like average trip time, number of passholders, etc...) and use our model to forecast revenue for the upcoming week.  


# Example 2

Let's predict the overflow rate (number of bikes taken from the station - number of bikes docked at the station) at certain stations on any given day based on weather conditions like Temperature. 

### API Call
Lets start by pulling in data from a particular start station (by Temperature)<br/>
https://ibm-wdai.looker.com/looks/10
and an end station (by Temperature) <br/>
https://ibm-wdai.looker.com/looks/11


### Model
Let's build a Linear regression model that predicts the number of trips from a station and to a station. 

In [36]:
#### Simple Linear Regression w/ Start Stations 

# Import Stats packages 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

# Step 1
# Set API Credentials for Demo Looker Instance and make an API call to get query results. 
api_endpoint = 'https://ibm-wdai.looker.com:19999/api/3.0/'
login_url = 'https://ibm-wdai.looker.com:19999/login'
client_id = 'vdRvj8tp8qSC4Yyt3ttj'
client_secret = 'gVYw9wQXY6n85fJCqyf5ZBqc'

demo = LookerAPI(api_endpoint, client_id, client_secret, login_url)
# Login to Instance - Return token to be used in subsequent calls.
try:
    json_auth = demo.login()
    start_station = demo.run_query_slug("KDE803l02xeZKIVNdUeuwp", json_auth)
    end_station = demo.run_query_slug("5Rptglr5g0uiFyVFLkEDfP", json_auth)
except:
    logger.warn("exception logging in and/or running query!")
    
# Convert Query Response from CSV to a dateframe
start_station = pd.read_csv(StringIO(start_station))
start_data = pd.DataFrame(start_station)
print(start_data.head())

end_station = pd.read_csv(StringIO(end_station))
end_data = pd.DataFrame(end_station)
print(end_data.head())


# Step 2
# Calculate best fit line for trips starting from Station
try:
    column_list = list(start_data)
    if(len(column_list) > 2 or len(column_list) < 6):
        print("Columns: " + str(column_list))
    
    column_list = list(end_data)
    if(len(column_list) > 2 or len(column_list) < 6):
        print("Columns: " + str(column_list))
except:
    logger.error("Your Query doesn't seem right. Recreate Query in Looker.")


## L-reg by Station using StatsModel
l_reg_start = []
df_group = start_data.groupby('Start Station Station ID')
for station, group in start_data.groupby('Start Station Station ID'):
    new_df= df_group.get_group(station)
    x=new_df['Weather Temperature (F)']
    y=new_df['Trip Trip Count']
    lm = smf.ols(formula='y ~ x', data=new_df).fit()
    l_reg_start.append({
        'bike_station': station, 
        'start_slope': lm.params.x,
        'start_intercept': lm.params.Intercept
    })

l_reg_start_df = pd.DataFrame(l_reg_start)  

print(l_reg_start_df.head())


# Step 3

## L-reg by End Station using StatsModel
l_reg_end = []
df_group = end_data.groupby('Trip To Station ID')
for station, group in end_data.groupby('Trip To Station ID'):
    new_df= df_group.get_group(station)
    x=new_df['Weather Temperature (F)']
    y=new_df['Trip Trip Count']
    lm = smf.ols(formula='y ~ x', data=new_df).fit()
    l_reg_end.append({
        'bike_station': station, 
        'end_slope': lm.params.x,
        'end_intercept': lm.params.Intercept
        
    })
l_reg_end_df = pd.DataFrame(l_reg_end)  
print(l_reg_end_df.head())


# Step 4
# Combine Start and End predictions by Bike Station
l_reg_df = pd.merge(l_reg_start_df, l_reg_end_df, on='bike_station')
l_reg_df.head()



Query_id: 232
Query_id: 228
  Trip Start Date Start Station Station ID  Weather Temperature (F)  \
0      2016-08-31                   SLU-19                       65   
1      2016-08-31                   SLU-07                       65   
2      2016-08-31                    BT-04                       65   
3      2016-08-31                    UW-04                       65   
4      2016-08-31                    CH-02                       65   

   Trip Trip Count  
0               10  
1                4  
2                8  
3                3  
4                7  
  Trip Start Date Trip To Station ID  Weather Temperature (F)  Trip Trip Count
0      2016-08-31             SLU-04                       65               15
1      2016-08-31              UW-02                       65                1
2      2016-08-31             DPD-03                       65                2
3      2016-08-31              EL-01                       65                6
4      2016-08-31       

,bike_station,start_intercept,start_slope,end_intercept,end_slope
0,BT-01,16.617737,-0.010712,-27.794702,0.556291
1,BT-03,6.156569,0.053592,6.553281,0.003612
2,BT-04,4.954260,0.039200,1.500763,0.125191
3,BT-05,-0.887119,0.131580,-10.845802,0.288550
4,CBD-03,3.264151,0.031925,25.900060,-0.311860


In [40]:
from ingest.Connectors import Connectors
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

dashdbsaveoption = {
Connectors.DASHDB.HOST : credentials["host"],
Connectors.DASHDB.DATABASE : credentials["database"],
Connectors.DASHDB.USERNAME : credentials["username"],
Connectors.DASHDB.PASSWORD : credentials["password"],
Connectors.DASHDB.TARGET_TABLE_NAME : 'BIKE_TRIPS.STATION_REGRESSION',
Connectors.DASHDB.TARGET_TABLE_ACTION : 'replace',
Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' }

l_reg_df2 = sqlContext.createDataFrame(l_reg_df)
NewdashDBDF = l_reg_df2.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

Now that we have our predictive model, Let's pull it up in Looker and calculate how Trips per station. 


Join in *station prediction* table in LookMl to our Trips Explore and create fields to predict trips taken based on Temperature. This quickly allows us to explore the data with predicted station counts in place. 

```
explore: trip {
  join: station_prediction  {
    type:  left_outer
    relationship: one_to_one
    sql_on:  ${trip.from_station_id} = ${station_prediction.bike_station} ;;
  }
```

**Predictive Fields **
Our predictive measure follows the linear regression model that we used and pulls in the coefficients and intercept that we just persisted back in to the DB.

**Linear Regression Model:** <br/>
$y = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$ <br/>
Each $x$ represents a different feature, and each feature has its own coefficient. <br/>
In our case: <br/>
$y = \beta_0 + \beta_1 \times Temperature$

We build these calculations in our LookML model 

```
  measure:  start_predictions {
    type:  average
    sql: (${start_slope} * ${weather.temperature} ) +  ${start_intercept};;
    value_format_name: decimal_1
  }

  measure:  end_predictions {
    type:  average
    sql: (${end_slope} * ${weather.temperature} ) +  ${end_intercept};;
    value_format_name: decimal_1
  }

  measure:  predicted_station_overflow {
    type: number
    sql:  ${end_predictions} - ${start_predictions} ;;
    value_format_name: decimal_1
  }
  
  
  
```



** Reports and Dashboards in Looker:** <br/>

Let's use our model to forecast trips taken from a station for future dates based on the temperature:
https://ibm-wdai.looker.com/looks/9

Let's bring in additional data about stations and plot overflow by Location for a particular date: 
Let's use our model to forecast trips taken from a station for future dates based on the temperature:
https://ibm-wdai.looker.com/looks/12

Pulling it all together we build a station manager dashboard, so a manager can take immediate action based on the predicted overflow at stations in their region: https://ibm-wdai.looker.com/dashboards/7

